In [1]:
library(tidyverse)
library(magrittr)
library(purrr) 
library(glue) 
library(stringr) 

library(rvest) 
library(polite)
library(xml2) 

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.8     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.1
✔ readr   2.1.2     ✔ forcats 0.5.1
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘magrittr’


The following object is masked from ‘package:purrr’:

    set_names


The following object is masked from ‘package:tidyr’:

    extract



Attaching package: ‘rvest’


The following object is masked from ‘package:readr’:

    guess_encoding




In [2]:
all_region_fire_incident <- read_html('https://www.fireandemergency.nz/incidents-and-news/incident-reports/')

In [3]:
north_region_href <- all_region_fire_incident %>%
   html_nodes('div.incidentreport__region:nth-child(6) .region .region__link') %>%
   html_attr('href') 
north_region_href

[1] "/incidents-and-news/incident-reports/incidents?region=1&day=Wednesday"
[2] "/incidents-and-news/incident-reports/incidents?region=1&day=Tuesday"  
[3] "/incidents-and-news/incident-reports/incidents?region=1&day=Monday"   
[4] "/incidents-and-news/incident-reports/incidents?region=1&day=Sunday"   
[5] "/incidents-and-news/incident-reports/incidents?region=1&day=Saturday" 
[6] "/incidents-and-news/incident-reports/incidents?region=1&day=Friday"   
[7] "/incidents-and-news/incident-reports/incidents?region=1&day=Thursday"

In [4]:
central_region_href <- all_region_fire_incident %>%
   html_nodes('div.incidentreport__region:nth-child(7) .region .region__link') %>%
   html_attr('href') 
central_region_href

[1] "/incidents-and-news/incident-reports/incidents?region=2&day=Wednesday"
[2] "/incidents-and-news/incident-reports/incidents?region=2&day=Tuesday"  
[3] "/incidents-and-news/incident-reports/incidents?region=2&day=Monday"   
[4] "/incidents-and-news/incident-reports/incidents?region=2&day=Sunday"   
[5] "/incidents-and-news/incident-reports/incidents?region=2&day=Saturday" 
[6] "/incidents-and-news/incident-reports/incidents?region=2&day=Friday"   
[7] "/incidents-and-news/incident-reports/incidents?region=2&day=Thursday"

In [5]:
south_region_href <- all_region_fire_incident %>%
   html_nodes('div.incidentreport__region:nth-child(8) .region .region__link') %>%
   html_attr('href') 
south_region_href

[1] "/incidents-and-news/incident-reports/incidents?region=3&day=Wednesday"
[2] "/incidents-and-news/incident-reports/incidents?region=3&day=Tuesday"  
[3] "/incidents-and-news/incident-reports/incidents?region=3&day=Monday"   
[4] "/incidents-and-news/incident-reports/incidents?region=3&day=Sunday"   
[5] "/incidents-and-news/incident-reports/incidents?region=3&day=Saturday" 
[6] "/incidents-and-news/incident-reports/incidents?region=3&day=Friday"   
[7] "/incidents-and-news/incident-reports/incidents?region=3&day=Thursday"

In [6]:
all_region_href <- tibble(North = north_region_href, Central = central_region_href, South = south_region_href)
all_region_href

North,Central,South
<chr>,<chr>,<chr>
/incidents-and-news/incident-reports/incidents?region=1&day=Wednesday,/incidents-and-news/incident-reports/incidents?region=2&day=Wednesday,/incidents-and-news/incident-reports/incidents?region=3&day=Wednesday
/incidents-and-news/incident-reports/incidents?region=1&day=Tuesday,/incidents-and-news/incident-reports/incidents?region=2&day=Tuesday,/incidents-and-news/incident-reports/incidents?region=3&day=Tuesday
/incidents-and-news/incident-reports/incidents?region=1&day=Monday,/incidents-and-news/incident-reports/incidents?region=2&day=Monday,/incidents-and-news/incident-reports/incidents?region=3&day=Monday
/incidents-and-news/incident-reports/incidents?region=1&day=Sunday,/incidents-and-news/incident-reports/incidents?region=2&day=Sunday,/incidents-and-news/incident-reports/incidents?region=3&day=Sunday
/incidents-and-news/incident-reports/incidents?region=1&day=Saturday,/incidents-and-news/incident-reports/incidents?region=2&day=Saturday,/incidents-and-news/incident-reports/incidents?region=3&day=Saturday
/incidents-and-news/incident-reports/incidents?region=1&day=Friday,/incidents-and-news/incident-reports/incidents?region=2&day=Friday,/incidents-and-news/incident-reports/incidents?region=3&day=Friday
/incidents-and-news/incident-reports/incidents?region=1&day=Thursday,/incidents-and-news/incident-reports/incidents?region=2&day=Thursday,/incidents-and-news/incident-reports/incidents?region=3&day=Thursday


In [7]:
region_incident_data <- function(href, key_index){
    
    region_data <- glue('https://www.fireandemergency.nz{href}#')%>%
                   read_html() %>%
                   html_nodes(glue('.report__table .report__table__body .report__table__row:nth-child({key_index}) .report__table__cell+ .report__table__cell--value p:nth-child(1)')) %>%
                   html_text2()
    
    return(region_data)
    
    }
                  

In [8]:
region_incident_data <- function(href){
    
    region_href <- glue('https://www.fireandemergency.nz{href}#')%>%
                   read_html() 
    
    Incident <- region_href %>%
                  html_nodes(glue('.report__table .report__table__body .report__table__row:nth-child(1) .report__table__cell+ .report__table__cell--value p:nth-child(1)')) %>%
                  html_text2()
    
    Date_and_Time <- region_href %>%
                  html_nodes(glue('.report__table .report__table__body .report__table__row:nth-child(2) .report__table__cell+ .report__table__cell--value p:nth-child(1)')) %>%
                  html_text2()
        
    Location <- region_href %>%
                  html_nodes(glue('.report__table .report__table__body .report__table__row:nth-child(3) .report__table__cell+ .report__table__cell--value p:nth-child(1)')) %>%
                  html_text2()
    
    Duration <- region_href %>%
                  html_nodes(glue('.report__table .report__table__body .report__table__row:nth-child(4) .report__table__cell+ .report__table__cell--value p:nth-child(1)')) %>%
                  html_text2()
    
    Station <- region_href %>%
                  html_nodes(glue('.report__table .report__table__body .report__table__row:nth-child(5) .report__table__cell+ .report__table__cell--value p:nth-child(1)')) %>%
                  html_text2()
    
    Result <- region_href %>%
                  html_nodes(glue('.report__table .report__table__body .report__table__row:nth-child(6) .report__table__cell+ .report__table__cell--value p:nth-child(1)')) %>%
                  html_text2()
    
    region_incident_table <- tibble(Incident_no = Incident, Date_and_Time = Date_and_Time, Location = Location, Duration = Duration, Station = Station, Result = Result)
    
    return(region_incident_table)
    
    }

format:
{region}_{day}_data

### North fire incident data (Monday to Sunday)

In [9]:
north_monday_data <- region_incident_data('/incidents-and-news/incident-reports/incidents?region=1&day=Monday')

In [10]:
north_tuesday_data <- region_incident_data('/incidents-and-news/incident-reports/incidents?region=1&day=Tuesday')

In [11]:
north_wednesday_data <- region_incident_data('/incidents-and-news/incident-reports/incidents?region=1&day=Wednesday')

In [12]:
north_thursday_data <- region_incident_data('/incidents-and-news/incident-reports/incidents?region=1&day=Thursday')

In [13]:
north_friday_data <- region_incident_data('/incidents-and-news/incident-reports/incidents?region=1&day=Friday')

In [14]:
north_saturday_data <- region_incident_data('/incidents-and-news/incident-reports/incidents?region=1&day=Saturday')

In [15]:
north_sunday_data <- region_incident_data('/incidents-and-news/incident-reports/incidents?region=1&day=Sunday')

### Central fire incident data (Monday to Sunday)

In [16]:
central_monday_data <- region_incident_data('/incidents-and-news/incident-reports/incidents?region=2&day=Monday')

In [17]:
central_tuesday_data <- region_incident_data('/incidents-and-news/incident-reports/incidents?region=2&day=Tuesday')

In [18]:
central_wednesday_data <- region_incident_data('/incidents-and-news/incident-reports/incidents?region=2&day=Wednesday')

In [19]:
central_thursday_data <- region_incident_data('/incidents-and-news/incident-reports/incidents?region=2&day=Thursday')

In [20]:
central_friday_data <- region_incident_data('/incidents-and-news/incident-reports/incidents?region=2&day=Friday')

In [21]:
central_saturday_data <- region_incident_data('/incidents-and-news/incident-reports/incidents?region=2&day=Saturday')

In [22]:
central_sunday_data <- region_incident_data('/incidents-and-news/incident-reports/incidents?region=2&day=Sunday')

### South fire incident data (Monday to Sunday)

In [23]:
south_monday_data <- region_incident_data('/incidents-and-news/incident-reports/incidents?region=3&day=Monday')

In [24]:
south_tuesday_data <- region_incident_data('/incidents-and-news/incident-reports/incidents?region=3&day=Tuesday')

In [25]:
south_wednesday_data <- region_incident_data('/incidents-and-news/incident-reports/incidents?region=3&day=Wednesday')

In [26]:
south_thursday_data <- region_incident_data('/incidents-and-news/incident-reports/incidents?region=3&day=Thursday')

In [27]:
south_friday_data <- region_incident_data('/incidents-and-news/incident-reports/incidents?region=3&day=Friday')

In [28]:
south_saturday_data <- region_incident_data('/incidents-and-news/incident-reports/incidents?region=3&day=Saturday')

In [29]:
south_sunday_data <- region_incident_data('/incidents-and-news/incident-reports/incidents?region=3&day=Sunday')